In [1]:
%load_ext autoreload
%autoreload 2
import os
os.environ["CUDA_VISIBLE_DEVICES"]="-1"
from qlora import *
from collections import defaultdict
import copy
import json
from os.path import exists, join, isdir
from dataclasses import dataclass, field
import sys
from typing import Optional, Dict, Sequence
import numpy as np
from tqdm import tqdm
import logging
import bitsandbytes as bnb
import pandas as pd
import importlib
from packaging import version
from packaging.version import parse
import warnings
from sklearn.metrics.pairwise import manhattan_distances
from torchmetrics.functional.pairwise import pairwise_manhattan_distance as manhattan
from torchmetrics.functional.pairwise import pairwise_cosine_similarity as cossim
import numpy as np

import torch
import transformers
from torch.nn.utils.rnn import pad_sequence
import argparse
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    set_seed,
    Seq2SeqTrainer,
    BitsAndBytesConfig,
    LlamaTokenizer

)
from datasets import load_dataset, Dataset, load_from_disk
import evaluate

from peft import (
    prepare_model_for_kbit_training,
    LoraConfig,
    get_peft_model,
    PeftModel
)
from peft.tuners.lora import LoraLayer
from transformers.trainer_utils import PREFIX_CHECKPOINT_DIR

from transformers.modeling_utils import unwrap_model
from transformers.models.auto.modeling_auto import MODEL_FOR_CAUSAL_LM_MAPPING_NAMES
from transformers.utils import is_peft_available
from peft import PeftModel

The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
/home/sonia/transformers-4.39.3/src/transformers/deepspeed.py:23: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(


In [2]:
argdict = {
  'model_name_or_path' : '/mnt/data/sonia/ckpts/clozehead/checkpoint-60', #'./mhllama',
  'num_heads' : 4,
  'data_seed' : 42 ,
  'do_eval': True,
  'eval_dataset_size' : 5 ,
  'max_eval_samples' : 2 ,
  'per_device_eval_batch_size' : 1 ,
  'max_new_tokens' : 60 ,
  'dataloader_num_workers' : 1 ,
  'group_by_length' : True,
  'remove_unused_columns' : False ,
  'lora_r' : 64 ,
  'lora_alpha' : 16 ,
  'lora_modules' : 'all' ,
  'double_quant' : True,
  'quant_type' : 'nf4' ,
  'bf16' : True,
  'bits' : 4 ,
  'dataset' : '/mnt/data/sonia/honeygan/apr23.dat',
  'source_max_len' : 60 ,
  'target_max_len' : 60 ,
  'seed' : 0
}

arglist = [f'--{k}={v}' for k,v in argdict.items()]

In [3]:
hfparser = transformers.HfArgumentParser((
    ModelArguments, DataArguments, TrainingArguments, GenerationArguments
))
model_args, data_args, training_args, generation_args  = hfparser.parse_args_into_dataclasses(args=arglist, return_remaining_strings=True)[:-1]
training_args.generation_config = transformers.GenerationConfig(**vars(generation_args))
args = argparse.Namespace(
    **vars(model_args), **vars(data_args), **vars(training_args)
)

# Eval

In [4]:
checkpoint_dir, completed_training = get_last_checkpoint(args.output_dir)
model, tokenizer = get_accelerate_model(args, checkpoint_dir)
model.config.use_cache = False
    
print('loaded model')
set_seed(args.seed)

loading base model /mnt/data/sonia/ckpts/clozehead/checkpoint-60...


/home/sonia/transformers-4.39.3/src/transformers/configuration_utils.py:363: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
/mnt/data/sonia/miniconda3/envs/llama/lib/python3.11/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of MultiheadLlamaForCausalLM were not initialized from the model checkpoint at /home/sonia/llama-qlora/mhllama and are newly initialized: ['heads.1.weight', 'heads.2.weight', 'heads.3.weigh

MH llama has 4 heads
Adding special tokens.
adding LoRA modules...
['base_layer']
loaded model


In [9]:
tokenizer = AutoTokenizer.from_pretrained('/mnt/data/zoo/llama2/llama2-7b-hf/',
        padding_side="right",
        use_fast=False, # Fast tokenizer giving issues.
        )

In [11]:
data_module = make_data_module(tokenizer=tokenizer, args=args)
collator = data_module['data_collator']
datatr = data_module['train_dataset']

In [24]:
trainerclass = Seq2SeqTrainer
trainer = trainerclass(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    **{k:v for k,v in data_module.items() if k != 'predict_dataset'},
)
class evalSampleCallback(transformers.TrainerCallback):
    def on_evaluate(self, args, state, control, model, **kwargs):
        trainer.model.eval()
        metrics = trainer.predict(test_dataset=data_module['eval_dataset'],metric_key_prefix="predict")
        
        predictions = []
        for i in range(len(metrics.predictions)):
            logit = metrics.predictions[i]
            label = metrics.label_ids[i] #just to see positions where prompt tokens are at
            logit_abcd = logit[label != IGNORE_INDEX]
            toks = np.argmax(logit_abcd, axis=1)
            predictions.append(
                ''.join(trainer.tokenizer.decode(toks, skip_special_tokens=True, clean_up_tokenization_spaces=True))
                )
        
        for pred in predictions:
            print(pred)
    
    
trainer.add_callback(evalSampleCallback)

/mnt/data/sonia/miniconda3/envs/llama/lib/python3.11/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [11]:
trainer.evaluate()

in forward. input_ids tensor([[  263,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     3,   472,  5641,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     3, 29892,  2011,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     3, 29892, 27032,  2669,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     3]], device='cuda:0') attention mask tensor([[ True, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False,  True,  True,  True,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False,  True,  True,  True, False,
         False, False, F

in forward. input_ids tensor([[  263,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     3,   472,  5641,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     3, 29892,  2011,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     3, 29892, 27032,  2669,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     3]], device='cuda:0') attention mask tensor([[ True, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False,  True,  True,  True, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False,  True,  True,  True, False, False, False,
         False, False, False, False, False, False, F

{'eval_loss': 12.072885513305664,
 'eval_runtime': 0.7302,
 'eval_samples_per_second': 2.739,
 'eval_steps_per_second': 2.739}

In [109]:
trainer.model.set_trace(True)

# Generation

In [5]:
transformers.AutoConfig.register('mhllama', MHLlamaConfig)
transformers.AutoModelForCausalLM.register(MHLlamaConfig, MultiheadLlamaForCausalLM)

In [6]:
config = MHLlamaConfig(**vars(args))
model = AutoModelForCausalLM.from_pretrained(
            args.model_name_or_path,
            config = config,)

/home/sonia/transformers-4.39.3/src/transformers/configuration_utils.py:363: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
/mnt/data/sonia/miniconda3/envs/llama/lib/python3.11/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [8]:
tokenizer = AutoTokenizer.from_pretrained('/mnt/data/zoo/llama2/llama2-7b-hf/',
        padding_side="right",
        use_fast=False, # Fast tokenizer giving issues.
        )
data_module = make_data_module(tokenizer=tokenizer, args=args)
collator = data_module['data_collator']

In [17]:
inputs = collator([{'os_generic':'synology diskstation manager (dsm)', 'ip_str':'60.250.84.90', 'port_str':'80', 'module':'http', 'length': 87}])
inputs
model.generate(inputs['input_ids'].cpu(), inputs['insert_indices'].cpu(), 14, )

/home/sonia/llama-qlora/multihead_models.py:453: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  if has_default_max_length and generation_config.max_new_tokens is None:
Input length of input_ids is 68, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.


in mlm_sample
in prepare inputs for generation with args
torch.Size([1, 68])
<class 'NoneType'>
<class 'torch.Tensor'>
<class 'NoneType'>
<class 'torch.Tensor'>
{'use_cache': True}
position_ids tensor([[ 0,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  2,  3,
          1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  4,  5,  6,  1,
          1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  7,  8,  9, 10,  1,
          1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, 11]])
position_ids tensor([[ 0,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  2,  3,
          1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  4,  5,  6,  1,
          1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  7,  8,  9, 10,  1,
          1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, 11]])
in forward. input_ids tensor([[  263,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,  6213,   472,  5641,     0, 

Traceback (most recent call last):
  File "/mnt/data/sonia/miniconda3/envs/llama/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3505, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_6290/3891907605.py", line 3, in <module>
    model.generate(inputs['input_ids'].cpu(), inputs['insert_indices'].cpu(), 14, )
  File "/mnt/data/sonia/miniconda3/envs/llama/lib/python3.11/site-packages/torch/utils/_contextlib.py", line 115, in decorate_context
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/home/sonia/llama-qlora/multihead_models.py", line 510, in generate
    return self.mlm_sample(
           ^^^^^^^^^^^^^^^^
  File "/home/sonia/llama-qlora/multihead_models.py", line 661, in mlm_sample
    next_token_logits = outputs.logits[:, insert_indices[1:], :] # batch_size x num_heads x vocab_size
                        ~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^
IndexError: index 41 is out of bounds for dimension 

# Normal llama

In [2]:
from transformers import AutoTokenizer, LlamaForCausalLM

model = LlamaForCausalLM.from_pretrained("/mnt/data/zoo/llama2/llama2-7b-hf/")
tokenizer = AutoTokenizer.from_pretrained("/mnt/data/zoo/llama2/llama2-7b-hf/")

prompt = "Hey, are you conscious? Can you talk to me?"
inputs = tokenizer(prompt, return_tensors="pt")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/home/sonia/transformers-4.39.3/src/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/sonia/transformers-4.39.3/src/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/sonia/transformers-4.39.3/src/transformers/generation/configuration_utils.py:4

In [7]:
# Generate
generate_ids = model.generate(inputs.input_ids, max_length=30)
tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

in generate None None None
in prepare inputs for generation with args
torch.Size([1, 13])
<class 'NoneType'>
<class 'torch.Tensor'>
<class 'NoneType'>
<class 'torch.Tensor'>
{'use_cache': True}
position_ids tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12]])
position_ids tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12]])
in forward
in prepare inputs for generation with args
torch.Size([1, 14])
<class 'tuple'>
<class 'torch.Tensor'>
<class 'NoneType'>
<class 'torch.Tensor'>
{'use_cache': True}
past_length 13 cache_length 13 max_cache_length None
input_ids case 2 before torch.Size([1, 14])
input_ids case 2 after  torch.Size([1, 1])
position_ids tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13]])
position_ids tensor([[13]])
in forward
in prepare inputs for generation with args
torch.Size([1, 15])
<class 'tuple'>
<class 'torch.Tensor'>
<class 'NoneType'>
<class 'torch.Tensor'>
{'use_cache': True}
past_length 14 cache_length 14 max_cache_length None
inp

In [41]:
%reload_ext autoreload